In [ ]:
from google.colab import drive
import os

MOUNT = True

if MOUNT:
  drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
FILEPATH = "/content/gdrive/MyDrive/HackYeah 2023 - dane/ML/raw"


In [ ]:
data = pd.read_csv(f'{FILEPATH}/states_all.csv')
# data = pd.read_csv(f'{FILEPATH}/parsed.csv')

# features = ['TOTAL_REVENUE', 'TOTAL_EXPENDITURE', 'GRADES_8_G', 'AVG_MATH_8_SCORE', 'AVG_READING_8_SCORE', 'DIFFERENCE', 'EXPENDITURE_PER_STUDENT', 'REVENUE_PER_STUDENT', 'REVENUE_EXPENDITURE_RATIO']
# scaled_data = data[features]


In [ ]:
data = data.sort_values(by=['STATE', 'YEAR'])

data['Expenditure_ROC'] = data.groupby('STATE')['TOTAL_EXPENDITURE'].pct_change()
data['Average_Score'] = data[['AVG_MATH_8_SCORE', 'AVG_READING_8_SCORE']].mean(axis=1)
data['Score_ROC'] = data.groupby('STATE')['Average_Score'].pct_change()


In [ ]:
data['ROC_Difference'] = data['Expenditure_ROC'] - data['Score_ROC']


In [ ]:
import matplotlib.pyplot as plt

def plot_trends(state_name):
    state_data = data[data['STATE'] == state_name]

    plt.figure(figsize=(12, 6))
    plt.plot(state_data['YEAR'], state_data['Expenditure_ROC'], label='Expenditure Rate of Change', marker='o')
    plt.plot(state_data['YEAR'], state_data['Score_ROC'], label='Score Rate of Change', marker='o')
    # plt.plot(state_data['YEAR'], state_data['AVG_MATH_8_SCORE'], label='Math', marker='o')
    plt.title(f'Trends for {state_name}')
    plt.xlabel('Year')
    plt.ylabel('Rate of Change')
    plt.legend()
    plt.grid(True)
    plt.show()

# Example usage
for state in data['STATE'].unique().tolist():
  plot_trends(state)  # Replace 'California' with any state in your dataset


In [ ]:
# Setting a threshold for ROC_Difference
threshold = 0.1

# Flagging schools where ROC_Difference exceeds the threshold
flagged_schools = data[data['ROC_Difference'] > threshold]

print(flagged_schools[['STATE', 'YEAR', 'Expenditure_ROC', 'Score_ROC', 'ROC_Difference']])


        STATE  YEAR  Expenditure_ROC  Score_ROC  ROC_Difference
1279  ALABAMA  1990         0.000000 -17.823507       17.823507
1280  ALABAMA  1991         0.000000  -1.059441        1.059441
0     ALABAMA  1992         0.878574 -18.471635       19.350209
51    ALABAMA  1993        -0.032451  -1.057236        1.024784
306   ALABAMA  1998        -0.066594 -10.482736       10.416142
...       ...   ...              ...        ...             ...
968   WYOMING  2010        -0.005821  -0.870290        0.864469
1070  WYOMING  2012        -0.002124  -0.909929        0.907804
1172  WYOMING  2014        -0.015432  -0.917700        0.902268
1274  WYOMING  2016        -0.014699  -0.893684        0.878985
1714  WYOMING  2019         0.000000  -0.573479        0.573479

[899 rows x 5 columns]
